In [16]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Instalação das bibliotecas
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rpjun\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rpjun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rpjun\AppData\Roaming\nltk_data...


True

In [5]:
df_1 = pd.read_csv('https://raw.githubusercontent.com/Rogeriom49/sexism_analysis/refs/heads/main/datasets/dev.csv')
df_2 = pd.read_csv('https://raw.githubusercontent.com/Rogeriom49/sexism_analysis/refs/heads/main/datasets/test%20(1).csv')
df_3 = pd.read_csv('https://raw.githubusercontent.com/Rogeriom49/sexism_analysis/refs/heads/main/datasets/train%20(2).csv')
df = pd.concat([df_1, df_2, df_3])

In [6]:
df.head()

,rewire_id,text,label_sexist,label_category,label_vector,split
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",not sexist,none,none,dev
1,sexism2022_english-966,I bet she wished she had a gun,not sexist,none,none,dev
2,sexism2022_english-3553,I agree with that but at the same time I know ...,sexist,2. derogation,2.1 descriptive attacks,dev
3,sexism2022_english-13328,democrats from Minnesota (leftist muzzie shith...,sexist,2. derogation,2.2 aggressive and emotive attacks,dev
4,sexism2022_english-81,Texas jury sentences Jordanian immigrant to de...,not sexist,none,none,dev


In [ ]:
# Criando a coluna label
df['label'] = df['label_sexist'].map({'not sexist': 0, 'sexist': 1})
df.head()

,rewire_id,text,label_sexist,label_category,label_vector,split,label
0,sexism2022_english-9609,"In Nigeria, if you rape a woman, the men rape ...",not sexist,none,none,dev,0
1,sexism2022_english-966,I bet she wished she had a gun,not sexist,none,none,dev,0
2,sexism2022_english-3553,I agree with that but at the same time I know ...,sexist,2. derogation,2.1 descriptive attacks,dev,1
3,sexism2022_english-13328,democrats from Minnesota (leftist muzzie shith...,sexist,2. derogation,2.2 aggressive and emotive attacks,dev,1
4,sexism2022_english-81,Texas jury sentences Jordanian immigrant to de...,not sexist,none,none,dev,0


In [ ]:
# Criando a coluna preprocessed_text
df_new = df[['text', 'label']]
df_new.head()

,text,label
0,"In Nigeria, if you rape a woman, the men rape ...",0
1,I bet she wished she had a gun,0
2,I agree with that but at the same time I know ...,1
3,democrats from Minnesota (leftist muzzie shith...,1
4,Texas jury sentences Jordanian immigrant to de...,0


In [ ]:
# Função para limpar o texto
def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]

    filtered_tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(filtered_tokens)

In [ ]:
# Aplicando a função
df_new['preprocessed_text'] = df_new['text'].apply(process_text)

In [18]:
df_new.head()

,text,label,preprocessed_text
0,"In Nigeria, if you rape a woman, the men rape ...",0,nigeria rape woman men rape back nsfw nigeria ...
1,I bet she wished she had a gun,0,bet wished gun
2,I agree with that but at the same time I know ...,1,agree time know well enough say love woman min...
3,democrats from Minnesota (leftist muzzie shith...,1,democrat minnesota leftist muzzie shithole dum...
4,Texas jury sentences Jordanian immigrant to de...,0,texas jury sentence jordanian immigrant death ...


In [ ]:
# Criando o vetorizador
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_new['preprocessed_text'])
y = df_new['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Criando o modelo
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# Testando o modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.90      0.87      3028
           1       0.60      0.45      0.52       972

    accuracy                           0.79      4000
   macro avg       0.72      0.68      0.69      4000
weighted avg       0.78      0.79      0.78      4000

0.79475
